## Battle Of the Neighborhoods - Applied Data Science Capstone
#### Final Assignment

In [1]:
#Importing all necessary libraries
import pandas as pd
from pandas import json_normalize
import numpy as np
import scipy
import folium
from bs4 import BeautifulSoup
import requests
import geocoder
from geopy.geocoders import Nominatim 
import geopy.distance as gdis
from sklearn.cluster import KMeans, DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib
import json
import itertools


---
### Data acquisition and preparation
---

In [2]:
ItemsDict={'Smartwatch':'Electronics Store','External Hard Drive':'Electronics Store','Pen and pencil':'Office Supplies','Notebook':'Office Supplies', 'Groceries':'Supermarket','Couch':'Furniture', 'Desk Chair':'Furniture','New bed':'Office Supplies','steakpan':'Home Store','Cooking pot': 'Home Store','Dog bed':'Pet supplies', 'coffee machine': 'Home Store', 'dog food': 'Pet supplies', 'drill bits':'Home Store'}
ItemsDict

{'Smartwatch': 'Electronics Store',
 'External Hard Drive': 'Electronics Store',
 'Pen and pencil': 'Office Supplies',
 'Notebook': 'Office Supplies',
 'Groceries': 'Supermarket',
 'Couch': 'Furniture',
 'Desk Chair': 'Furniture',
 'New bed': 'Office Supplies',
 'steakpan': 'Home Store',
 'Cooking pot': 'Home Store',
 'Dog bed': 'Pet supplies',
 'coffee machine': 'Home Store',
 'dog food': 'Pet supplies',
 'drill bits': 'Home Store'}

In [3]:
#Items to be bought along with the category of store they might be part of:

Items_df=pd.DataFrame(list(ItemsDict.items()),columns=['Item', 'Store Category'])
Items_df.sort_values(by='Store Category', inplace=True)
Items_df


,Item,Store Category
0,Smartwatch,Electronics Store
1,External Hard Drive,Electronics Store
5,Couch,Furniture
6,Desk Chair,Furniture
8,steakpan,Home Store
9,Cooking pot,Home Store
11,coffee machine,Home Store
13,drill bits,Home Store
2,Pen and pencil,Office Supplies
3,Notebook,Office Supplies


### Now, we need to obtain the unique categories in order to search them with the Foursquare API

In [4]:
for i in Items_df['Store Category'].unique():
    print (i)

Electronics Store
Furniture
Home Store
Office Supplies
Pet supplies
Supermarket


##### Great! Only 6 different stores to visit And upon closer inspection, "Furniture" and "Home Store" can be merged into the same category as you'll see later

In [5]:
#Foursquare Credentials
CLIENT_ID = '532KKFQ4NDT0AAJHBMX0MMQD4FOL5PJKK2RTT0A4G1FYLQCW' # your Foursquare ID
CLIENT_SECRET = 'KQFGSE4XT3HXATRU2JUQ5HQXEORI3MVIHWO5NUZUXE1YDNQX' # your Foursquare Secret
VERSION = '20180605'


In [6]:
#Get the geographical information for Stockholm:
Address='Stockholm, Sweden'

geolocator=Nominatim(user_agent='Stockholm_Search')
location=geolocator.geocode(Address)
latitude=location.latitude
longitude=location.longitude
print('The coordinates for Stockholm are {}, {}.'.format(latitude, longitude))


The coordinates for Stockholm are 59.3251172, 18.0710935.


In [7]:
#Getting the categroies ID to search by category from the Foursquare API
catids=requests.get('https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)).json()




In [8]:
type(catids)

dict

In [9]:
catids_pre_df=pd.DataFrame.from_dict(catids)

In [10]:
catids_pre_df['response'][2][8]['categories'][27]

{'id': '4bf58dd8d48988d103951735',
 'name': 'Clothing Store',
 'pluralName': 'Clothing Stores',
 'shortName': 'Apparel',
 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_',
  'suffix': '.png'},
 'categories': [{'id': '4bf58dd8d48988d102951735',
   'name': 'Accessories Store',
   'pluralName': 'Accessories Stores',
   'shortName': 'Accessories',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_accessories_',
    'suffix': '.png'},
   'categories': []},
  {'id': '4bf58dd8d48988d104951735',
   'name': 'Boutique',
   'pluralName': 'Boutiques',
   'shortName': 'Boutique',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_boutique_',
    'suffix': '.png'},
   'categories': []},
  {'id': '4bf58dd8d48988d105951735',
   'name': 'Kids Store',
   'pluralName': 'Kids Stores',
   'shortName': 'Kids Store',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_kids_',
    'suffix': '.png'},
   'catego

In [11]:
catids_list=[]

In [12]:
for i in range(len(catids_pre_df['response'][2][8]['categories'])):
    if 'Electronics' in catids_pre_df['response'][2][8]['categories'][i]['name']:
        catids_list.append(catids_pre_df['response'][2][8]['categories'][i])
    elif 'Furniture' in catids_pre_df['response'][2][8]['categories'][i]['name'] or 'Home Store' in catids_pre_df['response'][2][8]['categories'][i]['name']:
        catids_list.append(catids_pre_df['response'][2][8]['categories'][i])
    elif 'Office Supplies' in catids_pre_df['response'][2][8]['categories'][i]['name']:
        catids_list.append(catids_pre_df['response'][2][8]['categories'][i])
    elif 'Pet Store' in catids_pre_df['response'][2][8]['categories'][i]['name']:
        catids_list.append(catids_pre_df['response'][2][8]['categories'][i])
    elif 'Food' in catids_pre_df['response'][2][8]['categories'][i]['name']:
        for j in range(len(catids_pre_df['response'][2][8]['categories'][i]['categories'])):
            if 'Supermarket' in catids_pre_df['response'][2][8]['categories'][i]['categories'][j]['name']:
                catids_list.append(catids_pre_df['response'][2][8]['categories'][i]['categories'][j])
        

In [13]:
catids_df=pd.DataFrame(catids_list) #Convert list of IDs to a dataframe
catids_df

,id,name,pluralName,shortName,icon,categories
0,4bf58dd8d48988d122951735,Electronics Store,Electronics Stores,Electronics,{'prefix': 'https://ss3.4sqi.net/img/categorie...,[]
1,52f2ab2ebcbc57f1066b8b46,Supermarket,Supermarkets,Supermarket,{'prefix': 'https://ss3.4sqi.net/img/categorie...,[]
2,4bf58dd8d48988d1f8941735,Furniture / Home Store,Furniture / Home Stores,Furniture / Home,{'prefix': 'https://ss3.4sqi.net/img/categorie...,"[{'id': '55888a5a498e782e3303b43a', 'name': 'L..."
3,4bf58dd8d48988d121951735,Paper / Office Supplies Store,Paper / Office Supplies Stores,Office Supplies,{'prefix': 'https://ss3.4sqi.net/img/categorie...,[]
4,4bf58dd8d48988d100951735,Pet Store,Pet Stores,Pet Store,{'prefix': 'https://ss3.4sqi.net/img/categorie...,[]


In [14]:
#We can drom the last 4 columns as we don't need them

catids_df.drop(['categories', 'icon', 'shortName', 'pluralName'], axis=1, inplace=True)

In [15]:
catids_df.shape

(5, 2)

In [16]:
#Function to obtain the category type of each venue

def get_category_type(row):
    categories_list = row['Venue Category']

        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['shortName']

#result_df_test['categories']=result_df.apply(get_category_type, axis=1)

In [17]:
#Define function to get nearby venues that match our categories:
def GetNearbyVenues(category, latitude, longitude, radius=50000):
    venues_list=[]
    
    for index in range(catids_df.shape[0]):
        print ('catid: {},  Category: {}'.format(catids_df['id'][index], catids_df['name'][index]))
        
        url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET,
        latitude,
        longitude,
        VERSION,
        catids_df['id'][index],
        radius)
        
        res=requests.get(url).json()['response']['venues']
        print(type(res))
        print(len(res))
        
        venues_list.append([(a['name'], 
                             a['id'],
                            a['location']['lat'],
                            a['location']['lng'],
                            a['categories']) for a in res ])
        venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        venues.columns=['Venue Name','Venue ID', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(venues)

In [18]:
list(Items_df['Store Category'].unique())

['Electronics Store',
 'Furniture',
 'Home Store',
 'Office Supplies',
 'Pet supplies',
 'Supermarket']

In [19]:
Venues_df_test=GetNearbyVenues(list(Items_df['Store Category'].unique()), latitude, longitude)

catid: 4bf58dd8d48988d122951735,  Category: Electronics Store
<class 'list'>
30
catid: 52f2ab2ebcbc57f1066b8b46,  Category: Supermarket
<class 'list'>
30
catid: 4bf58dd8d48988d1f8941735,  Category: Furniture / Home Store
<class 'list'>
30
catid: 4bf58dd8d48988d121951735,  Category: Paper / Office Supplies Store
<class 'list'>
29
catid: 4bf58dd8d48988d100951735,  Category: Pet Store
<class 'list'>
29


In [20]:
#print(Venues_df_test.shape[0])
cat_list=[]
for number in range(0,Venues_df_test.shape[0]):
    if len(Venues_df_test['Venue Category'][number])>0:
        #print (number)
        #print(Venues_df_test['Venue Category'][number][0]['name'])
        continue
    else:
        Venues_df_test.drop(number, axis=0, inplace=True)

Venues_df_test.head(10)

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
5,Mi Store,5f4fd5828d5b390f8e0cd6c8,59.370520,18.003813,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
6,Kjell & Company,4bc1c42e4cdfc9b6a2059521,59.444982,18.071961,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
7,Webhallen,5853b8895d6ec60f2369781f,59.379182,18.025916,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
8,LAN-Master,4b62f157f964a520fd592ae3,59.340794,18.028154,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."
9,Webhallen,58396c84c0df761f5c0fb370,59.315329,18.071047,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E..."


In [21]:
Venues_df_test.reset_index(inplace=True) 
for i in range(0,Venues_df_test.shape[0]):
        Venues_df_test.loc[i,'Venue Category']=Venues_df_test.loc[i,'Venue Category'][0]['shortName']


In [22]:

Venues_df_test.reset_index(inplace=True)
Venues_df_test.drop(['level_0'], axis=1, inplace=True)
Venues_df_test.drop(['index'], axis=1, inplace=True)
Venues_df_test.head()

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,Electronics
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,Electronics
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,Electronics
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,Electronics


In [23]:
for i in Venues_df_test['Venue Category'].unique():
    print(i)

Electronics
Coworking Space
Video Games
Supermarket
Grocery Store
Furniture / Home
Gift Shop
Factory
Warehouse
Lighting
Restaurant
Design
Apparel
Hardware
Office Supplies
Bookstore
Construction
Arts & Crafts
Pet Store
Pet Service


#### We can see that there are several categories that clearly won't be of use for us. Let's drop them!

In [24]:
for i in range(0, Venues_df_test.shape[0]):
    if Venues_df_test['Venue Category'][i] not in ('Office Supplies','Electronics', 'Furniture / Home',
                                                   'Pet Store', 'Supermarket', 'Grocery Store'):
        Venues_df_test.drop(i, axis=0, inplace=True)
Venues_df_test.shape

(130, 5)

#### Let's further explore our remaining venues

In [25]:
Venues_df_test.reset_index(inplace=True)
Venues_df_test.drop(['index'], axis=1, inplace=True)
Venues_df_test.head()

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,Electronics
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,Electronics
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,Electronics
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,Electronics


#### We can merge grocery Store and supermarket for convenience

In [26]:
Venues_df_test['Venue Category'].value_counts()

Electronics         28
Pet Store           26
Supermarket         25
Office Supplies     24
Furniture / Home    22
Grocery Store        5
Name: Venue Category, dtype: int64

In [27]:
#Merging similar categories
for i in range(0,Venues_df_test.shape[0]):
    if Venues_df_test.loc[i,'Venue Category']=='Grocery Store' or Venues_df_test.loc[i,'Venue Category']=='Convenience Store':
        Venues_df_test.loc[i,'Venue Category']='Supermarket'
    elif Venues_df_test.loc[i,'Venue Category']=='Paper / Office Supplies Store' or Venues_df_test.loc[i,'Venue Category']=='Stationery Store':
        Venues_df_test.loc[i,'Venue Category']='Office Supplies'
    elif Venues_df_test.loc[i,'Venue Category']=='Furniture / Home Store':
        Venues_df_test.loc[i,'Venue Category']='Furniture'
        

In [28]:
Venues_df_test.sort_values(by='Venue Category')

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics
27,Elgiganten,4b0695e1f964a52073ed22e3,59.336137,18.068694,Electronics
26,NetOnNet,4bb47f0649bdc9b6bcd20c10,59.403568,17.843932,Electronics
25,Elgiganten Megastore,4b5dca01f964a520c66b29e3,59.305671,18.130570,Electronics
23,Elgiganten Megastore,4c4ee873f53d0f47be654ba7,59.421855,17.854537,Electronics
...,...,...,...,...,...
31,Stora Coop,4b1a76a8f964a520b4ea23e3,59.293682,18.004472,Supermarket
30,Willys Knivsta,55bb5a67498e998df113a929,59.727842,17.787507,Supermarket
29,Hemköp,4c231e7899282d7f0d2968b0,58.946256,17.486738,Supermarket
42,Coop,4b740d86f964a52099c52de3,59.263849,18.040098,Supermarket


In [29]:
Venues_df_test.reset_index(inplace=True)
Venues_df_test.drop(['index'], axis=1, inplace=True)
Venues_df_test.head()

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,Electronics
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,Electronics
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,Electronics
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,Electronics


In [30]:
Venues_df_test['Venue Category'].value_counts()


Supermarket         30
Electronics         28
Pet Store           26
Office Supplies     24
Furniture / Home    22
Name: Venue Category, dtype: int64

## Now let's plot the venues!

In [31]:
venues_map=folium.Map(location=[latitude, longitude], zoom_start=10) #Generate the map around Stockholm
Color_dict={'Electronics':'blue',  'Supermarket':'green', 'Furniture / Home':'yellow', 'Pet Store':'red', 'Office Supplies' : 'magenta'}
for lat, lng, label, cat in zip(Venues_df_test['Venue Latitude'], Venues_df_test['Venue Longitude'], Venues_df_test['Venue Name'], 
                                       Venues_df_test['Venue Category']):
    folium.CircleMarker([lat, lng],
                  radius=5, 
                  popup=label,
                  fill=True,
                  color=Color_dict[cat],
                  fill_color=Color_dict[cat],
                  fill_opacity=0.5).add_to(venues_map)

venues_map

### Now, lets start by finding the distance from the start point to each one of the venues

In [32]:
#Calculating distance to each venue from the starting point using geopy 

Venues_df_test['Distance to Start']=0 #Creating new column
for i in range(Venues_df_test.shape[0]):
    Venues_df_test.loc[i, 'Distance to Start']=gdis.distance((latitude, longitude) ,(Venues_df_test.loc[i, 'Venue Latitude'], Venues_df_test.loc[i,'Venue Longitude']))
    

In [33]:
Venues_df_test.sort_values(by='Distance to Start').tail(20)

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start
63,Folkhemmet,4d8f2c683bfef04dbf77a886,59.463886,18.123283,Furniture / Home,15.740913916825372 km
98,Office Depot,4daaf31d5da3ba8a4795c2f4,59.463784,18.129359,Office Supplies,15.798536403015694 km
76,IKEA,4b3f69d5f964a52089a625e3,59.421107,17.859065,Furniture / Home,16.113809525137622 km
124,Anderssons Häst & Hund,4eeb1f0c99119cf9cdbf3570,59.468268,18.126055,Pet Store,16.250082831869463 km
23,Elgiganten Megastore,4c4ee873f53d0f47be654ba7,59.421855,17.854537,Electronics,16.36191809772153 km
52,ICA Maxi,4b963063f964a520e9bf34e3,59.237425,17.832479,Supermarket,16.746830858925176 km
14,Netonnet,5970c0edd41bb7405a7122b6,59.176107,18.147850,Electronics,17.167658618620404 km
21,Webhallen,5561d0a3498e52484931dca6,59.493843,17.924769,Electronics,20.551416989602053 km
71,Mio InfraCity,4efdc31e9adffb90caf8dc2f,59.498599,17.923027,Furniture / Home,21.076135645978184 km
91,kontorab ab,59940fb63d479116acd20f3d,59.485693,18.276476,Office Supplies,21.35597419053461 km


#### Hmmm... we don't have much options to form the clusters when distance >16.5km, so let's also drops those, then re-plot venues in previous map

In [34]:
for i in range(Venues_df_test.shape[0]):
    if Venues_df_test['Distance to Start'][i]>16.5:
        Venues_df_test.drop(i, axis=0, inplace=True)

In [35]:
Venues_df_test.reset_index(inplace=True)
Venues_df_test.drop(['index'], axis=1, inplace=True)
Venues_df_test.head()

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.6184864534132789 km
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,Electronics,10.607211560136879 km
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,Electronics,11.936643493725187 km
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,Electronics,9.220411019997766 km
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,Electronics,3.2153152301815244 km


#### Let's try creating clusters by distance from the start point

In [36]:
Venues_df_test['Distance to Start']=Venues_df_test['Distance to Start'].astype('str').str[:-3].astype(float)
Venues_df_test.head(10)

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.618486
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,Electronics,10.607212
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,Electronics,11.936643
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,Electronics,9.220411
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,Electronics,3.215315
5,Mi Store,5f4fd5828d5b390f8e0cd6c8,59.370520,18.003813,Electronics,6.343136
6,Kjell & Company,4bc1c42e4cdfc9b6a2059521,59.444982,18.071961,Electronics,13.353242
7,Webhallen,5853b8895d6ec60f2369781f,59.379182,18.025916,Electronics,6.548295
8,LAN-Master,4b62f157f964a520fd592ae3,59.340794,18.028154,Electronics,3.003936
9,Webhallen,58396c84c0df761f5c0fb370,59.315329,18.071047,Electronics,1.090456


In [37]:
kclusters = 5

Venue_distance_clustering =Venues_df_test[['Venue Latitude','Venue Longitude','Distance to Start']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,n_init=1000, random_state=0).fit(Venue_distance_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 0, 0, 4, 3, 2, 3, 4, 1, 3, 3, 4, 2, 2, 1, 0, 3, 1, 1, 2, 2,
       2, 4, 2, 1, 1, 4, 4, 1, 4, 4, 3, 4, 4, 1, 4, 1, 3, 4, 0, 3, 2, 2,
       3, 1, 4, 4, 3, 3, 1, 3, 1, 1, 3, 0, 0, 2, 3, 1, 4, 1, 3, 1, 2, 1,
       0, 1, 1, 2, 2, 3, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 4, 0, 4,
       1, 2, 1, 4, 2, 3, 1, 1, 1, 4, 2, 2, 1, 0, 2, 4, 3, 2, 4, 0, 0, 3,
       4, 4, 2, 1, 0])

In [38]:
Venues_df_test['Cluster Label']=kmeans.labels_

In [39]:
Venues_df_test

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start,Cluster Label
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.618486,1
1,Kjell & Company,592d2d17a423627b6514e260,59.266931,17.923742,Electronics,10.607212,0
2,Kjell & Company,4b64556bf964a52052ab2ae3,59.362560,17.874540,Electronics,11.936643,0
3,Webhallen,54957924498eb41f84b3ebbe,59.242876,18.089323,Electronics,9.220411,0
4,Webhallen,56e7d404498e53ae0a1f099d,59.309788,18.023252,Electronics,3.215315,4
...,...,...,...,...,...,...,...
110,Arken Zoo,4b793ddaf964a52070f02ee3,59.349186,18.004807,Pet Store,4.628278,4
111,Simbi,4bf7c3405317a5937287fe7e,59.305289,18.126883,Pet Store,3.869553,4
112,Anderssons Häst & Hund,4eeb1f0c99119cf9cdbf3570,59.468268,18.126055,Pet Store,16.250083,2
113,Häst & Hund Huset,4c6fcfb1d97fa143b3f9f3ca,59.335913,18.065673,Pet Store,1.241616,1


### Let's plot the clusters!

In [40]:
Clusters_map=folium.Map(location=[latitude, longitude], zoom_start=10) #Generate the map around Stockholm
Clusters_dict={0:'blue',  1:'green', 2:'magenta', 3:'red', 4: 'yellow', 5:'cyan'}
for lat, lng, label, dist,clabel in zip(Venues_df_test['Venue Latitude'], Venues_df_test['Venue Longitude'], Venues_df_test['Venue Name'], 
                                       Venues_df_test['Distance to Start'], Venues_df_test['Cluster Label']):
    folium.CircleMarker([lat, lng],
                  radius=5, 
                  popup='Name: {} \n Distance: {:.2f} km \n Cluster Label: {}'.format(label, dist, clabel),
                  fill=True,
                  color=Clusters_dict[clabel],
                  fill_color=Clusters_dict[clabel],
                  fill_opacity=0.5).add_to(Clusters_map)
Clusters_map

### After this, we will use DBSCAN to find the places with the highest stores density! 

In [44]:
Venues_preDM_list=[] #Creating a list of dataframes with the initial cluster label
for clabel in Venues_df_test.sort_values('Cluster Label')['Cluster Label'].unique():
    Venues_preDM_list.append(Venues_df_test.loc[Venues_df_test['Cluster Label']==clabel].drop(['Venue Category','Venue ID', 'Venue Name','Distance to Start', 'Cluster Label', 'Venue Name'], axis=1).set_index(Venues_df_test['Venue ID'].loc[Venues_df_test[ 'Cluster Label']==clabel]))
#Venues_preDM_df=Venues_df_test.drop(['Venue Category','Venue ID', 'Venue Name','Distance to start', 'Cluster Label', 'Venue Name'], axis=1).set_index(Venues_df_test['Venue ID'] ) #Line commented adter first iteration

In [45]:
for i in range(len(Venues_preDM_list)):
    dbscanclus=Venues_preDM_list[i]
    dbscanclus=StandardScaler().fit_transform(dbscanclus)
    db=DBSCAN(eps=0.5, min_samples=5).fit(dbscanclus)
    Venues_preDM_list[i]['DBSCAN label']=db.labels_

In [46]:
i=1
while i<len(Venues_preDM_list):
    Venues_preDM_list[i].loc[Venues_preDM_list[i]['DBSCAN label']!=-1, 'DBSCAN label']=Venues_preDM_list[i].loc[Venues_preDM_list[i]['DBSCAN label']!=-1, 'DBSCAN label']+(max(Venues_preDM_list[i-1]['DBSCAN label'])+1)
    i+=1

In [47]:
dataframethatmightwork=Venues_preDM_list[0].append([Venues_preDM_list[1], Venues_preDM_list[2], Venues_preDM_list[3], Venues_preDM_list[4]]) #Putting each KMeans cluster division back together

In [48]:
Venues_df_test['DBSCAN label']=np.nan

In [49]:
for venueid in dataframethatmightwork.index:
    Venues_df_test.loc[Venues_df_test['Venue ID']==venueid, 'DBSCAN label']=dataframethatmightwork.loc[venueid, 'DBSCAN label']

In [50]:
#Creating colormap for plotting DBSCAN clusters
colors=plt.get_cmap('gist_rainbow')(np.linspace(0.0,1.0,len(dataframethatmightwork['DBSCAN label'].unique()))) 
colorsrgb=[]
for i in range(len(colors)):
    colorsrgb.append(matplotlib.colors.rgb2hex(colors[i][:3]))

In [51]:
dbscan_map=folium.Map(location=[latitude, longitude], zoom_start=10) #Generate the map around Stockholm

for lat, lng, label, dlabel in zip(Venues_df_test['Venue Latitude'], Venues_df_test['Venue Longitude'], Venues_df_test['Venue Name'] ,Venues_df_test['DBSCAN label']):
    c=((['#A9A9A9']) if dlabel == -1 else colorsrgb[int(dlabel)])
    folium.CircleMarker([lat, lng],
                  radius=5, 
                  popup='Name: {} \n Cluster Label: {}'.format(label, dlabel),
                  fill=True,
                  color=c,
                  fill_color=c,
                  fill_opacity=0.5).add_to(dbscan_map)
dbscan_map

In [52]:
V_Cat_set=set(Venues_df_test['Venue Category']) #Set of the categories needed. Used to selec clusters with all categories and ensure one venue per category in cluster

In [53]:
#Let's see how many clusters we have that contain all the categories we need
for i in Venues_df_test['DBSCAN label'].unique():
    if len(set(Venues_df_test.loc[Venues_df_test['DBSCAN label']==i, 'Venue Category']) &V_Cat_set)==5:
        print('DBSCAN label:{} \n Number of Categories: {} \n'.format(i, len(set(Venues_df_test.loc[Venues_df_test['DBSCAN label']==i, 'Venue Category']) &V_Cat_set)))
    

DBSCAN label:1.0 
 Number of Categories: 5 

DBSCAN label:-1.0 
 Number of Categories: 5 

DBSCAN label:6.0 
 Number of Categories: 5 



In [59]:
#The cluster with label '-1' is for the venues that did not land in a cluster (the noise part of DBSCAN). So let's put the other two clusters on a dataframe 
Final_Clusters_df=Venues_df_test.loc[Venues_df_test['DBSCAN label']==1]
Final_Clusters_df=Final_Clusters_df.append(Venues_df_test.loc[Venues_df_test['DBSCAN label']==6])
Final_Clusters_df

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start,Cluster Label,DBSCAN label
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.618486,1,1.0
25,Elgiganten,4b0695e1f964a52073ed22e3,59.336137,18.068694,Electronics,1.235228,1,1.0
37,Coop,4b51d0cdf964a520955627e3,59.339478,18.059821,Supermarket,1.723710,1,1.0
52,Lagerhaus,4b8e6536f964a520ac2033e3,59.331480,18.063670,Furniture / Home,0.825249,1,1.0
53,H&M Home,57e94df6498ed5e3a1155e54,59.332587,18.062661,Furniture / Home,0.960727,1,1.0
59,Lagerhaus,4dd659c2e4cd37c8938f9b18,59.335411,18.073797,Furniture / Home,1.157027,1,1.0
61,Norrgavel,4be3ed9c660ec9280828cd3b,59.337383,18.070127,Furniture / Home,1.367534,1,1.0
65,Sängvaruhuset SOVA,582ec891c7ec6b2419e219da,59.335943,18.070698,Furniture / Home,1.206213,1,1.0
67,Mio,4b51c8a5f964a520785527e3,59.334586,18.064641,Furniture / Home,1.116962,1,1.0
73,Ordning & Reda,54677ed0498e9e5bff8026a2,59.332605,18.068295,Office Supplies,0.849223,1,1.0


#### To reduce the number of duplicates and choose the better cluster, let's get each venue's score!

In [60]:
Venues_Ratings=[]
for venueid in Final_Clusters_df['Venue ID']:
    Venues_Ratings.append(requests.get('https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venueid,CLIENT_ID, CLIENT_SECRET, VERSION)).json())

In [61]:
Venues_Ratings[4]['response']['venue']['name']

'H&M Home'

In [62]:
Ratings_Dict={}
for i in range(len(Venues_Ratings)):
    try:
        temp={Venues_Ratings[i]['response']['venue']['id']:Venues_Ratings[i]['response']['venue']['rating'] }
        Ratings_Dict.update(temp)
    except:
        temp={Venues_Ratings[i]['response']['venue']['id']: 'No Rating'}
        Ratings_Dict.update(temp)

In [63]:
Final_Clusters_df['Venue Rating']=np.nan
for key in Ratings_Dict.keys():
    Final_Clusters_df.loc[Final_Clusters_df['Venue ID']==key, 'Venue Rating']=Ratings_Dict[key]
Final_Clusters_df

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start,Cluster Label,DBSCAN label,Venue Rating
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.618486,1,1.0,5.6
25,Elgiganten,4b0695e1f964a52073ed22e3,59.336137,18.068694,Electronics,1.235228,1,1.0,5.5
37,Coop,4b51d0cdf964a520955627e3,59.339478,18.059821,Supermarket,1.723710,1,1.0,6
52,Lagerhaus,4b8e6536f964a520ac2033e3,59.331480,18.063670,Furniture / Home,0.825249,1,1.0,8
53,H&M Home,57e94df6498ed5e3a1155e54,59.332587,18.062661,Furniture / Home,0.960727,1,1.0,No Rating
59,Lagerhaus,4dd659c2e4cd37c8938f9b18,59.335411,18.073797,Furniture / Home,1.157027,1,1.0,6.2
61,Norrgavel,4be3ed9c660ec9280828cd3b,59.337383,18.070127,Furniture / Home,1.367534,1,1.0,7.7
65,Sängvaruhuset SOVA,582ec891c7ec6b2419e219da,59.335943,18.070698,Furniture / Home,1.206213,1,1.0,No Rating
67,Mio,4b51c8a5f964a520785527e3,59.334586,18.064641,Furniture / Home,1.116962,1,1.0,5.3
73,Ordning & Reda,54677ed0498e9e5bff8026a2,59.332605,18.068295,Office Supplies,0.849223,1,1.0,No Rating


In [72]:
#Final_Clusters_df.reset_index(inplace=True, drop=True)
#Final_Clusters_df.rename({'Cluster Label':'KMeans Label'}, axis=1, inplace=True) #Just for added clarity
Final_Clusters_df

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start,KMeans Label,DBSCAN label,Venue Rating
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.618486,1,1.0,5.6
1,Elgiganten,4b0695e1f964a52073ed22e3,59.336137,18.068694,Electronics,1.235228,1,1.0,5.5
2,Coop,4b51d0cdf964a520955627e3,59.339478,18.059821,Supermarket,1.723710,1,1.0,6
3,Lagerhaus,4b8e6536f964a520ac2033e3,59.331480,18.063670,Furniture / Home,0.825249,1,1.0,8
4,H&M Home,57e94df6498ed5e3a1155e54,59.332587,18.062661,Furniture / Home,0.960727,1,1.0,No Rating
5,Lagerhaus,4dd659c2e4cd37c8938f9b18,59.335411,18.073797,Furniture / Home,1.157027,1,1.0,6.2
6,Norrgavel,4be3ed9c660ec9280828cd3b,59.337383,18.070127,Furniture / Home,1.367534,1,1.0,7.7
7,Sängvaruhuset SOVA,582ec891c7ec6b2419e219da,59.335943,18.070698,Furniture / Home,1.206213,1,1.0,No Rating
8,Mio,4b51c8a5f964a520785527e3,59.334586,18.064641,Furniture / Home,1.116962,1,1.0,5.3
9,Ordning & Reda,54677ed0498e9e5bff8026a2,59.332605,18.068295,Office Supplies,0.849223,1,1.0,No Rating


#### For cluster with label 6, there's no need to remove venues,as this cluster is complete as is. For cluster with label 1 we need to remove a few venues to have only one venue of each category. When available, the highest rated venue will be kept. Otherwise, one will be chosen arbitrarily to aviod further complexity

In [75]:
#Final_Clusters_df.drop([1,4,5,6,7,8,9,11,12,], axis=0, inplace=True)
#Final_Clusters_df.reset_index(drop=True, inplace=True)
Final_Clusters_df

,Venue Name,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Distance to Start,KMeans Label,DBSCAN label,Venue Rating
0,Media Markt,4ecb821eb6346e2d39bfc4a9,59.330468,18.068199,Electronics,0.618486,1,1.0,5.6
1,Coop,4b51d0cdf964a520955627e3,59.339478,18.059821,Supermarket,1.723710,1,1.0,6
2,Lagerhaus,4b8e6536f964a520ac2033e3,59.331480,18.063670,Furniture / Home,0.825249,1,1.0,8
3,Office Depot,4d3abef36de7721ea98cf149,59.336295,18.077113,Office Supplies,1.291500,1,1.0,No Rating
4,Häst & Hund Huset,4c6fcfb1d97fa143b3f9f3ca,59.335913,18.065673,Pet Store,1.241616,1,1.0,No Rating
5,Elgiganten,4bc34375abf495214f79c393,59.460833,18.133707,Electronics,15.531925,2,6.0,6.5
6,ICA Kvantum Arninge,4b7c245ff964a520a87f2fe3,59.462519,18.134148,Supermarket,15.720502,2,6.0,7.3
7,Folkhemmet,4d8f2c683bfef04dbf77a886,59.463886,18.123283,Furniture / Home,15.740914,2,6.0,No Rating
8,Office Depot,4daaf31d5da3ba8a4795c2f4,59.463784,18.129359,Office Supplies,15.798536,2,6.0,No Rating
9,Anderssons Häst & Hund,4eeb1f0c99119cf9cdbf3570,59.468268,18.126055,Pet Store,16.250083,2,6.0,No Rating


###### In order to simplify things, we will add the average of the existing ratings to the venues without ratings

In [77]:
mean_a=(5.6+6+8)/3
meana_b=(6.5+7.3)/2

In [79]:
Final_Clusters_df.loc[[3,4], 'Venue Rating']=mean_a
Final_Clusters_df.loc[[7,8,9], 'Venue Rating']=mean_b

##### Now, let's compute the mean rating and distance of each cluser!

In [90]:
#This step is not needed in this specific case, but if we had ratings for all of the venues from the beginning, this would substitute the previous step.

rating_1=Final_Clusters_df.loc[Final_Clusters_df['DBSCAN label']==1, 'Venue Rating'].mean()
rating_6=Final_Clusters_df.loc[Final_Clusters_df['DBSCAN label']==6, 'Venue Rating'].mean()

In [87]:
#Let's also calculate the average distance to the start of the cluster to develop a simple score to choose the best cluster!
dist_1=Final_Clusters_df.loc[Final_Clusters_df['DBSCAN label']==1, 'Distance to Start'].mean()
dist_6=Final_Clusters_df.loc[Final_Clusters_df['DBSCAN label']==6, 'Distance to Start'].mean()

#### And now let's choose a simple score to choose our cluster:

<center>$\left(0.7 \cdot R_{avg}+0.3 \cdot 10\cdot \left(1-\frac{Dist_{avg}}{Dist_{Max}}\right)\right)=\left(0.7 \cdot R_{avg}+3 \cdot \left(1-\frac{Dist_{avg}}{Dist_{Max}}\right)\right)$<\center>
    
A little bit more explanation will be given in the report! 

In [116]:
def Score(Rating, DAvg, DMax):
    s=0.8*Rating+(2*(1-(DAvg)/(DMax)))
    return s

In [117]:
s1=Score(rating_1, dist_1, 50)
s6=Score(rating_6, dist_6, 50)
print('Score for cluster 1: {} \nScore for cluster 6: {}'.format(s1, s6))

Score for cluster 1: 7.18106217560012 
Score for cluster 6: 6.887664317522728


## Now we can conclude that cluster number 1 will be the best for our purposes, which as seen in the last map with the DBSCAN clusters, happens to be also the closest! Great!!